# Enfoque de validación

Antes de pasar al ejercicio de predicción creamos los subconjuntos de datos nos van a servir como inputs para realizar la validación de cada modelo, y de esa forma poder evaluar el desempeño de cada estimador ante la predicción de nuevos datos, con el objetivo de minimizar el problema del sobreajuste.

Dicho esto, primero cargaremos las librerías necesarias y la base modificada en la sección de ingeniería de atributos. 

In [1]:
import pandas as pd 
import numpy as np # numpy y pandas to data wrangling 
from datetime import datetime, timedelta # to work w date
from google.colab import drive

In [2]:
aux = 'https://drive.google.com/file/d/1GVQaQzc8orPwL9-XSJ8Y2aV9D0XKjw51/view?usp=sharing'
link ='https://drive.google.com/uc?id=' + aux.split('/')[-2]
df = pd.read_csv(link)
# set fecha como índice 
df = df.set_index('Date')
df

,log_y_lag1,log_y_lag2,log_y_lag3,log_y_lag4,log_y_lag5,log_y_lag6,log_y_lag7,log_price_eth,log_price_btc,log_price_btc_lag1,...,inflation_adjusted_log,taxes_adjusted_log,digital_wallet_top_adjusted_log,covid19_adjusted_log,pandemic_adjusted_log,elon musk_adjusted_log,do kwon_adjusted_log,future_commodities_adjusted_log,banking_adjusted_log,foreign_currency_adjusted_log
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-17,0.401401,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211,0.401401,2.420566,2.411216,...,1.232488,1.632153,1.087071,0.000000,0.232996,0.212188,0.886491,1.474216,1.908699,1.554368
2015-08-18,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459,0.401401,2.401401,2.420566,...,1.278754,1.621280,1.028978,0.000000,0.206826,0.204120,0.991226,1.467608,1.888516,1.566791
2015-08-19,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425,0.361728,2.357935,2.401401,...,1.273233,1.632153,1.000434,0.000000,0.222716,0.243038,0.939519,1.493458,1.878062,1.578868
2015-08-20,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398,0.436163,2.371068,2.357935,...,1.261976,1.626751,1.121888,0.000000,0.184691,0.257679,1.021189,1.511883,1.883321,1.629410
2015-08-21,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398,0.335458,0.397940,2.369457,2.371068,...,1.244525,1.586925,1.028978,0.000000,0.212188,0.278754,0.886491,1.546543,1.883321,1.607669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,3.203747,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.221142,4.463774,4.478630,...,2.004321,1.522966,1.547775,0.738781,0.691965,0.477121,0.278754,1.707996,1.859978,1.937016
2022-06-11,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.185397,4.453471,4.463774,...,1.732394,1.382017,1.547775,0.704151,0.632457,0.477121,0.278754,1.596597,1.708931,1.824126
2022-06-12,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502,3.157361,4.424898,4.453471,...,1.672098,1.357554,1.502427,0.691965,0.673021,0.531479,0.278754,1.561340,1.631444,1.781037


Con la base cargada, pasamos a identificar las 7 variables target, que corresponden al precio futuro de ethereum de los 7 días futuros inmediatos al corriente. Con estos nombres identificados creamos el dataset *y* que contendrá solamente los targets y *X* las variables predictoras.

In [3]:
# Creación de dataframe con variables de interés + target
target_list = list(df.loc[:,('log_y_lag' in c for c in df.columns)].columns)
y = df.loc[:, target_list]
X = df.drop(target_list, axis=1)
# visualizamos el dataframe de predictoras
X

,log_price_eth,log_price_btc,log_price_btc_lag1,log_price_btc_lag2,log_price_btc_lag3,log_price_btc_lag4,log_price_btc_lag5,log_price_btc_lag6,log_price_btc_lag7,log_price_eth_lag1,...,inflation_adjusted_log,taxes_adjusted_log,digital_wallet_top_adjusted_log,covid19_adjusted_log,pandemic_adjusted_log,elon musk_adjusted_log,do kwon_adjusted_log,future_commodities_adjusted_log,banking_adjusted_log,foreign_currency_adjusted_log
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-17,0.401401,2.420566,2.411216,2.428135,2.424882,2.421604,2.425551,2.431782,2.421604,0.326336,...,1.232488,1.632153,1.087071,0.000000,0.232996,0.212188,0.886491,1.474216,1.908699,1.554368
2015-08-18,0.401401,2.401401,2.420566,2.411216,2.428135,2.424882,2.421604,2.425551,2.431782,0.401401,...,1.278754,1.621280,1.028978,0.000000,0.206826,0.204120,0.991226,1.467608,1.888516,1.566791
2015-08-19,0.361728,2.357935,2.401401,2.420566,2.411216,2.428135,2.424882,2.421604,2.425551,0.401401,...,1.273233,1.632153,1.000434,0.000000,0.222716,0.243038,0.939519,1.493458,1.878062,1.578868
2015-08-20,0.436163,2.371068,2.357935,2.401401,2.420566,2.411216,2.428135,2.424882,2.421604,0.361728,...,1.261976,1.626751,1.121888,0.000000,0.184691,0.257679,1.021189,1.511883,1.883321,1.629410
2015-08-21,0.397940,2.369457,2.371068,2.357935,2.401401,2.420566,2.411216,2.428135,2.424882,0.436163,...,1.244525,1.586925,1.028978,0.000000,0.212188,0.278754,0.886491,1.546543,1.883321,1.607669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,3.221142,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,4.472764,3.252827,...,2.004321,1.522966,1.547775,0.738781,0.691965,0.477121,0.278754,1.707996,1.859978,1.937016
2022-06-11,3.185397,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,3.221142,...,1.732394,1.382017,1.547775,0.704151,0.632457,0.477121,0.278754,1.596597,1.708931,1.824126
2022-06-12,3.157361,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,3.185397,...,1.672098,1.357554,1.502427,0.691965,0.673021,0.531479,0.278754,1.561340,1.631444,1.781037


In [4]:
# visualizamos nuestro dataset de targets
y

,log_y_lag1,log_y_lag2,log_y_lag3,log_y_lag4,log_y_lag5,log_y_lag6,log_y_lag7
Date,,,,,,,
2015-08-17,0.401401,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211
2015-08-18,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459
2015-08-19,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425
2015-08-20,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398
2015-08-21,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398,0.335458
...,...,...,...,...,...,...,...
2022-06-10,3.203747,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672
2022-06-11,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881
2022-06-12,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502


Luego, pasamos a dividir ambos dataset entre entrenamiento y test. Para ello determinamos como separador el 31 de diciembre de 2020. De esta forma, las observaciones previas a esta fecha serán los datos de entrenamiento, y las correspondientes a días posteriores serán parte del subconjunto de testeo.

In [5]:
start = f'{X.index[0]}' 
end = f'{X.index[0]}'
# Enfoque de validación
train = X[start:'2020-12-31']
test  = X['2021-01-01':]
y_train = y[start:'2020-12-31']
y_test  = y['2021-01-01':]
# visualizamos los datasets de predictores
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)

Train Dataset: (1956, 49)
Test Dataset: (536, 49)


De esta forma, 1956 observaciones se utilizarán para entrenar el modelo, y las restantes 536 serán para la evaluación final de los estimadores.

In [6]:
# Exportamos a google drive
drive.mount('drive')
# train
train.to_csv('train.csv')
!cp train.csv "/content/drive/MyDrive/ds/proyecto/dataset/"
# test
test.to_csv('test.csv')
!cp test.csv "/content/drive/MyDrive/ds/proyecto/dataset/"
# target en train
y_train.to_csv('y_train.csv')
!cp y_train.csv "/content/drive/MyDrive/ds/proyecto/dataset/"
# target en test
y_test.to_csv('y_test.csv')
!cp y_test.csv "/content/drive/MyDrive/ds/proyecto/dataset/"

Mounted at drive
